In [1]:
from FedMA.frcnn_helper import *
from FedMA.helper_cyjui import * 

from Nb_utils import *


NOTE! Installing ujson may make loading annotations faster.


/home/superorange5/.local/lib/python3.8/site-packages/torchvision/transforms/transforms.py:803: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [2]:
from pt.engine.trainer import PTrainer


In [3]:
cfg_path = "configs/FedMA/ck2b_FedMA.yaml"

cfg = setup_all(cfg_path)

In [4]:
model_path_list = cfg.MODEL.TEACHER_PATH  
teacher_trainer = cfg.MODEL.TEACHER_TRAINER


In [5]:
      
def load_FRCNNmodel_cpu(cfg, model_path): 
    print("load FRCNN model")
    Trainer =DefaultTrainer
    model = Trainer.build_model(cfg)    
    DetectionCheckpointer(model).resume_or_load(model_path, resume=False)
    return model.model.cpu()
def load_TSmodel_cpu(cfg, model_path):
    Trainer =PTrainer
    model = Trainer.build_model(cfg)
    model_teacher = Trainer.build_model(cfg)
    ensem_ts_model = EnsembleTSModel(model_teacher, model)    
    DetectionCheckpointer(ensem_ts_model).resume_or_load(model_path, resume=False)
    return ensem_ts_model.modelStudent.cpu()


def get_trainer(trainer_name, cfg, model_path):
    if trainer_name == "pt":
        return load_TSmodel_cpu(cfg, model_path)       
    elif trainer_name == "default":
        return load_FRCNNmodel_cpu(cfg, model_path)       
    else:
        raise ValueError("Trainer Name is not found.")


In [6]:
model_list=[ get_trainer(teacher_trainer, cfg, model_teacher_path) for model_teacher_path in model_path_list]

-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


In [7]:
model_list[1].proposal_generator.rpn_head.conv.weight[0]

tensor([[[-0.0070,  0.0113, -0.0005],
         [ 0.0097, -0.0141,  0.0025],
         [ 0.0073,  0.0043, -0.0015]],

        [[ 0.0075, -0.0011, -0.0116],
         [ 0.0009, -0.0015,  0.0118],
         [ 0.0025, -0.0094, -0.0083]],

        [[-0.0097,  0.0133,  0.0023],
         [-0.0035,  0.0033, -0.0105],
         [-0.0061, -0.0029, -0.0080]],

        ...,

        [[ 0.0035,  0.0007, -0.0001],
         [ 0.0021, -0.0235,  0.0012],
         [-0.0081,  0.0078,  0.0029]],

        [[ 0.0036,  0.0059, -0.0054],
         [ 0.0133, -0.0101, -0.0169],
         [-0.0041,  0.0101, -0.0267]],

        [[ 0.0061, -0.0048, -0.0020],
         [-0.0033, -0.0059,  0.0034],
         [-0.0009,  0.0052, -0.0022]]], grad_fn=<SelectBackward>)

In [8]:
### single pass 
assignments_list = []
matching_shapes  = []
num_workers = len(model_list)
nets, model_meta_data, layer_type = init_vgg16_rcnns(num_workers)

vgg_weights = pdm_prepare_weights_vggs([m.backbone for m in model_list], 'cpu')


In [9]:
NUM_VGG_LAYERS = 7
VGG_CONV3_IDX = 6

In [10]:
for vgg_layer_idx in range(1, NUM_VGG_LAYERS):
        
    if vgg_layer_idx < VGG_CONV3_IDX: ### fix layers before conv3.
        vgg_weights, assignments_list = BBP_MAP_trivial(vgg_weights,
                                                        assignments_list,
                                                        vgg_layer_idx)
        continue
        ### note that we don't retrain for freezed layers. 
    else:
        vgg_weights, assignments_list = BBP_MAP_VGG(vgg_weights,
                                                    assignments_list,
                                                    matching_shapes,
                                                    vgg_layer_idx,
                                                    model_meta_data, 
                                                    layer_type,
                                                    device='cpu')
        
        

!!!!!!! L_next : 259, layer_index: 6
===============assignment: 256, weights: (2304,)


In [11]:
### update model w/ matched VGG.

### end of VGG matching.
calc_matched_shape = lambda weights:[w.shape for w in weights]



In [12]:
backbone_dim = [calc_matched_shape(vgg_weights[0])[index][0] for index in list(range(0,len(calc_matched_shape(vgg_weights[0])), 2))]

In [13]:
backbone_dim

[64, 64, 128, 128, 256, 259, 256, 512, 512, 512, 512, 512, 512]

In [14]:
### substitute matched vggs into fasterRCNN. (note that the shape of vgg might differ)
matched_vggs    = [matched_vgg16_no_FC(calc_matched_shape(weights)) for weights in vgg_weights]
vgg_state_dicts = [reconst_weights_to_state_dict(w, matched) 
                   for (w, matched) in zip(vgg_weights, matched_vggs)]

for vgg, state_dict in zip(matched_vggs, vgg_state_dicts):
    vgg.load_state_dict(state_dict)



* feed vgg weight to detectron2

In [15]:
conv_index = [0,2,5,7,10,12,14,17,19,21,24,26,28]

In [16]:
fedma_vgg_key =[]
for conv_i in conv_index:
    fedma_vgg_key.append('features.{}.weight'.format(conv_i))
    fedma_vgg_key.append('features.{}.bias'.format(conv_i))

In [17]:
from pt.engine.trainer_sourceonly import PTrainer_sourceonly

Trainer= PTrainer_sourceonly
initial_backbone = Trainer.build_model(cfg,backbone_dim,False) 

detectron_vgg_key_map = []

for key,value in initial_backbone.backbone.state_dict().items():
    detectron_vgg_key_map.append(key)

In [18]:
import copy

initial_backbone_list = [copy.deepcopy(initial_backbone)for i in range(len(model_list))]

In [19]:
from collections import OrderedDict
for model_idx, model_initial in enumerate(model_list):
    new_fedma_dict = OrderedDict()
    #copy backbone part from fedma model
    for key_idx,key in enumerate(fedma_vgg_key): 
        detectron_key = detectron_vgg_key_map[key_idx]
        fedma_weight = vgg.state_dict()[key]
        new_fedma_dict['backbone.'+detectron_key] = fedma_weight
    #copy rpn part from original model
    for key,value in model_initial.state_dict().items():  
        if 'backbone' not in key:
            new_fedma_dict[key] = value
        
    initial_backbone_list[model_idx].load_state_dict(new_fedma_dict)  

In [20]:
initial_backbone_list[1].backbone.state_dict()['vgg_block4.0.conv1.weight'][0]


tensor([[[-1.4771e-02, -1.5764e-02, -1.4989e-02],
         [-1.8325e-02, -2.4207e-02, -2.0874e-02],
         [-1.4645e-02, -1.8770e-02, -1.3101e-02]],

        [[ 4.4393e-03,  2.0369e-03,  3.4120e-03],
         [ 3.8087e-03,  1.1182e-03,  2.1083e-03],
         [ 2.9618e-03,  8.5050e-04,  8.4720e-04]],

        [[ 5.6242e-03,  3.5747e-03, -2.0005e-03],
         [ 2.4485e-02,  1.6768e-02, -1.0118e-03],
         [ 1.8295e-02,  1.5545e-02,  5.1186e-03]],

        ...,

        [[ 5.5129e-03,  2.5303e-03,  8.0368e-04],
         [ 4.8418e-03,  4.0211e-03,  3.5152e-03],
         [ 7.5664e-03,  5.0465e-03,  8.8466e-03]],

        [[-3.7132e-04,  7.4335e-05, -4.9293e-03],
         [-4.8853e-03, -3.3454e-03, -5.3191e-03],
         [-5.4915e-04, -2.4545e-03, -4.2009e-03]],

        [[-6.6348e-03, -6.9641e-03, -3.3403e-03],
         [-4.2094e-03, -3.3828e-03, -1.2286e-03],
         [-4.6744e-03, -3.5226e-03, -5.9668e-03]]], device='cuda:0')

In [21]:
from shutil import copyfile

source_dataset_list = cfg.FEDSET.DATASET_LIST
output_folder = cfg.OUTPUT_DIR
copyfile(cfg_path, os.path.join(output_folder, 'cfg.yaml'))

for model_idx, model_after_fedma in enumerate(initial_backbone_list):            
    model_save_name  = os.path.join(output_folder,'FedMA_{}_{}.pth'.format(source_dataset_list[model_idx],vgg_layer_idx))
    torch.save(model_after_fedma.state_dict(), model_save_name)     

* freeze layer

In [22]:
from detectron2.layers import ShapeSpec
from pt.modeling.backbone.vgg import VGG, VGGBlock
from pt.modeling.backbone.vgg import build_vgg_backbone_dynamic 

In [23]:
source_dataset = source_dataset_list[0]
vgg_layer_idx=6

In [24]:
initial_model_path  = os.path.join(output_folder,'FedMA_{}_{}.pth'.format(source_dataset,vgg_layer_idx))

In [25]:
cfg.defrost()
cfg.MODEL.WEIGHTS = initial_model_path                
cfg.OUTPUT_DIR = os.path.join(output_folder,source_dataset+"_"+str(vgg_layer_idx))
print("output subdir={}".format(cfg.OUTPUT_DIR))
cfg.DATASETS.TRAIN_LABEL=source_dataset
print("current source={}".format(source_dataset))
cfg.BACKBONE_DIM = backbone_dim
cfg.freeze()
                

output subdir=./output/FedMA_ck2b_source-only/VOC2007_kittismall_6
current source=VOC2007_kittismall


In [26]:
cfg.FEDSET.DYNAMIC 


True

In [27]:
cfg.BACKBONE_DIM


[64, 64, 128, 128, 256, 259, 256, 512, 512, 512, 512, 512, 512]

In [30]:
from pt.engine.trainer_sourceonly import PTrainer_sourceonly

Trainer= PTrainer_sourceonly   
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)


In [48]:
freeze_layer_fedma(trainer.model.backbone,vgg_layer_idx)

In [47]:
def freeze_layer_fedma(VGG,vgg_layer_idx):
    conv_layer_list = [(1,0),(1,1),(2,0),(2,1),(3,0),(3,1),(3,2),(4,0),(4,1),(4,2),(5,0),(5,1)]
    freeze_tuple = conv_layer_list[vgg_layer_idx-1]
    freeze_block, freeze_layer = freeze_tuple
    for idx, (stage, _) in enumerate(VGG.stages_and_names, start=1):    
        if idx <freeze_block:
            for block in stage.children():
                    block.freeze()
        elif idx == freeze_block:
            for block in stage.children():        
                for i_idx, layer in enumerate(block.children()):
                    if i_idx <=freeze_layer:                
                        for p in layer.parameters():
                            p.requires_grad=False

In [9]:
input_shape = ShapeSpec(channels=len(cfg.MODEL.PIXEL_MEAN))

VGG_backbone = build_vgg_backbone_dynamic(cfg, input_shape, None, True)
VGG_backbone.freeze_by_layer((3,1))

-------- pretrained model loaded ---------


In [41]:
VGG = VGG(stages, out_features=out_features, pretrain=pretrain, load_pretrained=True).freeze(freeze_at)


-------- pretrained model loaded ---------


In [35]:
freeze_tuple

(3, 1)

In [107]:
freeze_tuple = (3,1)
freeze_block, freeze_layer = freeze_tuple

In [111]:

for idx, (stage, _) in enumerate(VGG.stages_and_names, start=1):    
    if idx <freeze_block:
        for block in stage.children():
                block.freeze()
    elif idx == freeze_block:
        for block in stage.children():        
            for i_idx, layer in enumerate(block.children()):
                if i_idx <=freeze_layer:                
                    for p in layer.parameters():
                        p.requires_grad=False
            

* check freeze

In [55]:
trainer.model.backbone.vgg_block3[0].conv2.weight.requires_grad

False

In [56]:
for idx, (stage, _) in enumerate(trainer.model.backbone.stages_and_names, start=1):    
    print("conv block {}".format(idx))
    for block in stage.children():        
        for i_idx, layer in enumerate(block.children()):
            print("layer: {}".format(i_idx))
            for p in layer.parameters():
                print(p.requires_grad)
            

conv block 1
layer: 0
False
False
layer: 1
False
False
layer: 2
conv block 2
layer: 0
False
False
layer: 1
False
False
layer: 2
conv block 3
layer: 0
False
False
layer: 1
False
False
layer: 2
True
True
layer: 3
conv block 4
layer: 0
True
True
layer: 1
True
True
layer: 2
True
True
layer: 3
conv block 5
layer: 0
True
True
layer: 1
True
True
layer: 2
True
True


In [18]:
cfg.MODEL.BACKBONE.FREEZE_AT

2

In [31]:
### Freeze the layers
for model, vgg in zip(model_list, matched_vggs):
    vgg_freeze_to(vgg.features, vgg_layer_idx)
    model.features = vgg.features
    model.cuda()

In [34]:
model.backbone

VGG(
  (vgg_block1): Sequential(
    (0): VGGBlock(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (vgg_block2): Sequential(
    (0): VGGBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (vgg_block3): Sequential(
    (0): VGGBlock(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, 